### Preprocessing the Data for a Random Forest Classifier

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from keras.utils import np_utils
from config import db_password
import psycopg2
import pandas as pd
import tensorflow as tf

In [3]:
# Import and read csv
# import pandas as pd 
# kushy_washington_cleaned_df = pd.read_csv("./Data/kushy_washington_cleaned.csv")
# print(kushy_washington_cleaned_df.shape)
# kushy_washington_cleaned_df.head()

In [4]:
# Connect to sql database
con = psycopg2.connect(database='Medical-Marijuana-Group', user = 'postgres', password = db_password, host = '127.0.0.1', port = '5432')

# Create a new cursor
cur = con.cursor()

print('Opened database successfully')

Opened database successfully


In [5]:
# Create function that takes in a PostgreSQL query and outputs a Pandas dataframe
def create_pandas_df(sql_query, database = con):
    df = pd.read_sql_query(sql_query, database)
    return df
  
# Utilize the create_pandas_df function to create a Pandas dataframe
# Store the data as a variable
kushy_washington_cleaned_df = create_pandas_df('SELECT * FROM kushy_washington_cleaned')
print(kushy_washington_cleaned_df.shape)
kushy_washington_cleaned_df.head()

(722, 36)


,test_id,test_strain,type,ailment_1,ailment_2,ailment_3,ailment_4,ailment_5,effects_1,effects_2,...,inventory_type,chemotype,strain_leafly_page_rank,strain_leafly_review_rank,strain_tests,strain_chemotype,strain_prop_chemotype1,strain_prop_chemotype2,strain_prop_chemotype3,strain_modal_chemo_prop
0,19,Dutch Hawaiian,Sativa,Depression,Stress,Pain,Nausea,None,Happy,Energetic,...,Flower Lot,1,619,141,425,1,1.0,0.0,0.0,1.0
1,21,SleeStack,Hybrid,Stress,Depression,None,None,None,Uplifted,Euphoric,...,Flower Lot,1,1276,151,63,1,1.0,0.0,0.0,1.0
2,22,Space Needle,Sativa,Depression,Stress,None,None,None,Uplifted,Euphoric,...,Flower Lot,1,0,0,0,0,0.0,0.0,0.0,0.0
3,23,Double Purple Doja,Hybrid,Pain,Stress,Inflammation,Muscle Spasms,None,Relaxed,Creative,...,Flower Lot,1,1115,151,52,1,1.0,0.0,0.0,1.0
4,37,Jack Skellington,Hybrid,Stress,Depression,Pain,Muscle Spasms,None,Creative,Relaxed,...,Flower Lot,1,708,141,124,1,1.0,0.0,0.0,1.0


In [6]:
# Close the cursor and connection so the server can allocate bandwidth to other requests
cur.close()
con.close()
print('Closed database successfully')

Closed database successfully


In [7]:
# Drop non-beneficial identification columns
kushy_washington_df = kushy_washington_cleaned_df.drop(['test_id', 'test_strain'], axis=1)
print(kushy_washington_df.shape)
kushy_washington_df.head()

(722, 34)


,type,ailment_1,ailment_2,ailment_3,ailment_4,ailment_5,effects_1,effects_2,effects_3,effects_4,...,inventory_type,chemotype,strain_leafly_page_rank,strain_leafly_review_rank,strain_tests,strain_chemotype,strain_prop_chemotype1,strain_prop_chemotype2,strain_prop_chemotype3,strain_modal_chemo_prop
0,Sativa,Depression,Stress,Pain,Nausea,None,Happy,Energetic,Uplifted,Euphoric,...,Flower Lot,1,619,141,425,1,1.0,0.0,0.0,1.0
1,Hybrid,Stress,Depression,None,None,None,Uplifted,Euphoric,Creative,Relaxed,...,Flower Lot,1,1276,151,63,1,1.0,0.0,0.0,1.0
2,Sativa,Depression,Stress,None,None,None,Uplifted,Euphoric,Happy,Hungry,...,Flower Lot,1,0,0,0,0,0.0,0.0,0.0,0.0
3,Hybrid,Pain,Stress,Inflammation,Muscle Spasms,None,Relaxed,Creative,Sleepy,Happy,...,Flower Lot,1,1115,151,52,1,1.0,0.0,0.0,1.0
4,Hybrid,Stress,Depression,Pain,Muscle Spasms,None,Creative,Relaxed,Euphoric,Energetic,...,Flower Lot,1,708,141,124,1,1.0,0.0,0.0,1.0


In [8]:
# Determine number of unique values in each column
kushy_washington_df.nunique()

type                           4
ailment_1                      9
ailment_2                      9
ailment_3                     10
ailment_4                     10
ailment_5                      9
effects_1                     14
effects_2                     23
effects_3                     24
effects_4                     26
effects_5                     28
effects_6                      4
effects_7                      4
effects_8                      4
flavor_1                      32
flavor_2                      50
flavor_3                      42
ailment_count                  6
effects_count                  9
flavor_count                   4
thc_max                      697
cbd_max                      207
thc_cbd                      558
cbd_thc                       48
inventory_type                 7
chemotype                      3
strain_leafly_page_rank      640
strain_leafly_review_rank    128
strain_tests                 291
strain_chemotype               4
strain_pro

In [9]:
# Check datatypes
kushy_washington_df.dtypes

type                          object
ailment_1                     object
ailment_2                     object
ailment_3                     object
ailment_4                     object
ailment_5                     object
effects_1                     object
effects_2                     object
effects_3                     object
effects_4                     object
effects_5                     object
effects_6                     object
effects_7                     object
effects_8                     object
flavor_1                      object
flavor_2                      object
flavor_3                      object
ailment_count                  int64
effects_count                  int64
flavor_count                   int64
thc_max                      float64
cbd_max                      float64
thc_cbd                      float64
cbd_thc                      float64
inventory_type                object
chemotype                      int64
strain_leafly_page_rank        int64
s

In [10]:
# Use labelencoder to encode target ailment_1 column and check datatype
le = LabelEncoder()
kushy_washington_df['ailment_1'] = le.fit_transform(kushy_washington_df['ailment_1'])
kushy_washington_df['ailment_1']

0      0
1      8
2      0
3      7
4      8
      ..
717    7
718    1
719    8
720    6
721    6
Name: ailment_1, Length: 722, dtype: int32

In [11]:
# Check datatypes
kushy_washington_df.dtypes

type                          object
ailment_1                      int32
ailment_2                     object
ailment_3                     object
ailment_4                     object
ailment_5                     object
effects_1                     object
effects_2                     object
effects_3                     object
effects_4                     object
effects_5                     object
effects_6                     object
effects_7                     object
effects_8                     object
flavor_1                      object
flavor_2                      object
flavor_3                      object
ailment_count                  int64
effects_count                  int64
flavor_count                   int64
thc_max                      float64
cbd_max                      float64
thc_cbd                      float64
cbd_thc                      float64
inventory_type                object
chemotype                      int64
strain_leafly_page_rank        int64
s

In [12]:
# Convert vector of integers to one hot encoding using keras function to_categorical
dummy_y = np_utils.to_categorical(kushy_washington_df['ailment_1'])
print(dummy_y.shape)

(722, 9)


In [13]:
# Use get_dummies() to encode object columns
kushy_washington_df_encoded = pd.get_dummies(kushy_washington_df, columns=None)
print(kushy_washington_df_encoded.shape)
kushy_washington_df_encoded.head()

(722, 317)


,ailment_1,ailment_count,effects_count,flavor_count,thc_max,cbd_max,thc_cbd,cbd_thc,chemotype,strain_leafly_page_rank,...,flavor_3_Tree,flavor_3_Tropical,flavor_3_Woody,inventory_type_Bubble Hash,inventory_type_CO2 Hash Oil,inventory_type_Flower Lot,inventory_type_Food Grade Solvent Extract,inventory_type_Hash,inventory_type_Hydrocarbon Wax,inventory_type_Marijuana Mix
0,0,4,7,2,20.1309,0.54,37.28,0.03,1,619,...,0,0,0,0,0,1,0,0,0,0
1,8,2,7,2,9.9139,0.00,0.00,0.00,1,1276,...,0,0,0,0,0,1,0,0,0,0
2,0,2,7,1,10.8132,0.00,0.00,0.00,1,0,...,0,0,0,0,0,1,0,0,0,0
3,7,4,7,2,11.6702,0.00,0.00,0.00,1,1115,...,0,0,0,0,0,1,0,0,0,0
4,8,4,8,2,13.4657,0.00,0.00,0.00,1,708,...,0,0,0,0,0,1,0,0,0,0


In [14]:
# Split preprocessed data into features and target arrays
X = kushy_washington_df_encoded.drop(['ailment_1'],axis=1)
y = dummy_y
# Split preprocessed data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=72)

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Create, Fit, Predict and Evaluate the Model

In [16]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Fit the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Make predictions using testing data
predictions = rf_model.predict(X_test_scaled)
predictions

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
# Calculate confusion matrix
cm = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

# Create a DataFrame from the confusion matrix
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1", "Actual 2", "Actual 3", "Actual 4", "Actual 5", "Actual 6", "Actual 7", "Actual 8"], 
    columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3", "Predicted 4", "Predicted 5", "Predicted 6", "Predicted 7", "Predicted 8"])
cm_df

,Predicted 0,Predicted 1,Predicted 2,Predicted 3,Predicted 4,Predicted 5,Predicted 6,Predicted 7,Predicted 8
Actual 0,19,0,0,0,0,0,0,0,1
Actual 1,0,0,0,0,0,0,0,0,1
Actual 2,13,0,0,0,0,0,0,0,0
Actual 3,2,0,0,0,0,0,0,0,0
Actual 4,1,0,0,0,0,0,0,0,0
Actual 5,2,0,0,0,0,0,0,0,0
Actual 6,0,0,0,0,0,0,56,0,0
Actual 7,20,0,0,0,0,0,0,5,2
Actual 8,4,0,0,0,0,0,1,0,54


In [18]:
# Calculate accuracy score
acc_score = accuracy_score(y_test, predictions)
acc_score

0.6795580110497238

In [19]:
# Display results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2,Predicted 3,Predicted 4,Predicted 5,Predicted 6,Predicted 7,Predicted 8
Actual 0,19,0,0,0,0,0,0,0,1
Actual 1,0,0,0,0,0,0,0,0,1
Actual 2,13,0,0,0,0,0,0,0,0
Actual 3,2,0,0,0,0,0,0,0,0
Actual 4,1,0,0,0,0,0,0,0,0
Actual 5,2,0,0,0,0,0,0,0,0
Actual 6,0,0,0,0,0,0,56,0,0
Actual 7,20,0,0,0,0,0,0,5,2
Actual 8,4,0,0,0,0,0,1,0,54


Accuracy Score : 0.6795580110497238
Classification Report
              precision    recall  f1-score   support

           0       0.73      0.40      0.52        20
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         2
           6       0.98      1.00      0.99        56
           7       1.00      0.19      0.31        27
           8       0.93      0.92      0.92        59

   micro avg       0.94      0.68      0.79       181
   macro avg       0.40      0.28      0.30       181
weighted avg       0.84      0.68      0.71       181
 samples avg       0.68      0.68      0.68       181



C:\Users\jemis\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jemis\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
# Calculate feature importance in Random Forest model
importances = rf_model.feature_importances_
importances

array([6.93715965e-02, 5.05861486e-02, 1.58750096e-02, 1.68281369e-02,
       1.39449923e-02, 1.36959370e-02, 7.79861104e-03, 1.51546698e-03,
       2.07642664e-02, 2.17558570e-02, 1.84981088e-02, 2.90579249e-03,
       8.76768389e-03, 5.72844415e-03, 1.78658594e-03, 7.52455314e-03,
       4.99123225e-03, 5.51874697e-03, 0.00000000e+00, 2.71933645e-03,
       2.76182059e-02, 1.43783195e-03, 6.16627673e-03, 1.63505390e-03,
       1.16943786e-03, 6.85524714e-04, 2.10369849e-02, 6.19357772e-02,
       4.17956647e-02, 1.03958248e-02, 2.24286830e-03, 7.23821943e-03,
       3.34250958e-03, 1.35437474e-03, 1.34029841e-03, 1.46354720e-02,
       7.66815615e-04, 1.94515042e-02, 4.67558300e-02, 7.70650722e-03,
       3.20033829e-03, 2.20158024e-03, 3.66748114e-03, 2.32216084e-03,
       1.57743131e-03, 3.41006058e-03, 7.91455108e-06, 5.03891095e-03,
       2.03361346e-02, 1.89831489e-03, 1.35232295e-03, 8.94264426e-04,
       2.20064327e-03, 2.41131508e-03, 1.37547102e-03, 2.28464829e-04,
      

In [22]:
# Sort features by importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.06937159647494517, 'ailment_count'),
 (0.061935777163072164, 'ailment_2_ Stress'),
 (0.05058614855143993, 'effects_count'),
 (0.04675583000813912, 'ailment_3_None'),
 (0.04179566473830515, 'ailment_2_None'),
 (0.0390944150169688, 'effects_6_None'),
 (0.027618205874609877, 'ailment_2_ Depression'),
 (0.021755856979705234, 'strain_leafly_review_rank'),
 (0.021036984911309946, 'ailment_2_ Pain'),
 (0.02076426642184178, 'strain_leafly_page_rank'),
 (0.020336134566692605, 'ailment_4_None'),
 (0.019451504184843885, 'ailment_3_ Stress'),
 (0.01851123145384119, 'effects_7_None'),
 (0.01849810881491075, 'strain_tests'),
 (0.01682813694684546, 'thc_max'),
 (0.015875009585930275, 'flavor_count'),
 (0.014923236097291874, 'effects_6_ Dry Mouth'),
 (0.0146354719814568, 'ailment_3_ Pain'),
 (0.013944992342167128, 'cbd_max'),
 (0.013695937015012203, 'thc_cbd'),
 (0.012211594400757309, 'effects_4_None'),
 (0.011616296716419512, 'ailment_5_None'),
 (0.010395824801997078, 'ailment_3_ Depression'),
 (